In [ ]:
import fnmatch
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from skopt import gbrt_minimize
from skopt.plots import plot_evaluations, plot_objective

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

np.random.seed(777)

In [ ]:
def load_data(filename, time_step=0.1):
    df = pd.read_csv(filename, dtype=np.float64)
    t = np.cumsum(np.ones((df.shape[0],))*time_step) - time_step
    left = df.left[0] - df.left
    right = df.right[0] - df.right
    return t, left, right

In [ ]:
def find_data():
    for root, dirs, files in os.walk("./data/"):
        for basename in files:
            if fnmatch.fnmatch(basename, "data*.csv"):
                yield os.path.join(root, basename)

Given is the following transfer function:

$$
    H(s) = \frac{K}{s (T s + 1)}
$$

Its step response in time domain is given by

$$
    g(t) = K T (e^{-\frac{1}{T} t} - 1) + K t
$$

In [ ]:
def exact_step_response(K, T):
    return lambda t: K*T*(np.exp(-t/T) - 1.0) + K*t

In [ ]:
def simple_cost(t, y, kappa, K, T):
    model = K*T*(np.exp(-t/T) - 1.0) + K*t
    sq_e = np.square(y - model*kappa)
    return np.sqrt(np.mean(sq_e))

In [ ]:
def cost_fun(x):
    cost = 0.0
    kappa = 50.0
    for filename in find_data():
        t, l, r = load_data(filename)
        cost += simple_cost(t, l, kappa, x[0], x[1])
        cost += simple_cost(t, r, kappa, x[0], x[1])
    return cost

In [ ]:
%%time

res = gbrt_minimize(
    cost_fun, 
    [(1.2, 2.2), (0.1, 2.0)], 
    n_calls=1000,
    )

In [ ]:
_ = plot_evaluations(res)
_ = plot_objective(res)

In [ ]:
res.x

In [ ]:
plt.figure(figsize=(12, 8))
for filename in find_data():
    t, l, r = load_data(filename)
    plt.plot(t, l, c="b", alpha=0.2)
    plt.plot(t, r, c="r", alpha=0.2)
    
t = np.linspace(0.0, 12.0, 500)
fun = exact_step_response(res.x[0], res.x[1])
plt.plot(t, fun(t)*50.0, c="k")
plt.show()